In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls

drive  MNIST  MNIST.tar.gz  MNIST.tar.gz.1  sample_data


In [ ]:
cd ..

/


In [ ]:
!ls

bin	 datalab  home	 lib64	opt   run   sys		       tools
boot	 dev	  lib	 media	proc  sbin  tensorflow-1.15.2  usr
content  etc	  lib32  mnt	root  srv   tmp		       var


In [ ]:
cd content/

/content


In [ ]:
!ls

drive  MNIST  MNIST.tar.gz  MNIST.tar.gz.1  sample_data


In [ ]:
cd /content/drive/MyDrive/AMMI_MNIST

/content/drive/MyDrive/AMMI_MNIST


In [ ]:
!ls

dlc_practical_prologue.py


In [ ]:
from dlc_practical_prologue import *
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, transforms as transform


In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

from torchvision.datasets import MNIST

train_data = MNIST(root = './', train=True, download=True, transform=transform)
val_data = MNIST(root = './', train=False, download=True, transform=transform)

--2021-03-19 21:51:26--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-19 21:51:26--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [  <=>               ]  33.20M  2.71MB/s    in 13s     

2021-03-19 21:51:40 (2.53 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [ ]:
train_input = train_data.data.view(-1, 1, 28, 28).float()
train_target = train_data.targets
test_input = val_data.data.view(-1, 1, 28, 28).float()
test_target = val_data.targets

train_input = train_input.clone().reshape(train_input.size(0), -1)
test_input = test_input.clone().reshape(test_input.size(0), -1)

train_input = train_input.narrow(0, 0, 500)
train_target = train_target.narrow(0, 0, 500)
test_input = test_input.narrow(0, 0, 100)
test_target = test_target.narrow(0, 0, 100)

train_target = convert_to_one_hot_labels(train_input, train_target)
test_target = convert_to_one_hot_labels(test_input, test_target)

mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)
test_input.sub_(mu).div_(std)

tensor([[-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        ...,
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167]])

In [ ]:
train_target *= 0.9
test_target *= 0.9 

In [ ]:
test_target.shape

torch.Size([100, 10])

In [ ]:
class NeuralNet:
  def __init__(self,input_size=784,
               hidden_state=50,
               output_size=10,
               lr=0.1):
    self.lr=lr
    self.w1 = torch.Tensor(input_size, hidden_state).normal_(0, 1e-6)
    self.b1 = torch.Tensor(1, hidden_state).normal_(0, 1e-6)
    self.w2 = torch.Tensor(hidden_state,output_size).normal_(0, 1e-6)
    self.b2 = torch.Tensor(1, output_size).normal_(0, 1e-6)

  def sigma(self, x):
      return torch.tanh(x)
  def dsigma(self, x):
    return 1 - torch.pow(torch.tanh(x), 2)
  def loss(self, v, t):
    return torch.sum(torch.pow(t - v, 2))
  def dloss(self, v, t):
    return 2.0 * (v - t)
  
  def forward_pass(self,x):
    x0=x
    s1=torch.mm(x0,self.w1) +self.b1
    x1 = self.sigma(s1)
    s2 = torch.mm(x1, self.w2) + self.b2
    x2 = self.sigma(s2)
    return x0, s1, x1, s2, x2

  def backward_pass(self,t,x,s1,x1,s2,x2):
    d = self.dloss(x2, t) * self.dsigma(s2)
    dl_dw2 = torch.mm(d.t(), x1).t()
    dl_db2 = d.mean(axis=0).unsqueeze(0)
    dl_dw1 = torch.mm(x.t(), (d @ self.w2.t() * self.dsigma(s1)))
    dl_db1 = (d @ self.w2.t() * self.dsigma(s1)).mean(axis=0).unsqueeze(0)

    self.w1 = self.w1 - (self.lr * dl_dw1)/t.shape[0]
    self.w2 = self.w2 - (self.lr * dl_dw2)/t.shape[0]
    self.b1 = self.b1 - (self.lr * dl_db1)/t.shape[0]
    self.b2 = self.b2 - (self.lr * dl_db2)/t.shape[0]

  def train(self,X,y,iters=1000):
    for i in range(iters):
      x,s1,x1,s2,x2=self.forward_pass(X)
      self.backward_pass(y,x,s1,x1,s2,x2)
      if i%100==0:
        print(f"At iter:{i},")
        print(f"loss ={self.loss(y,x2)} and")
        print(f"error_rate = {(y.argmax(axis=1) != x2.argmax(axis=1)).sum().item()/y.shape[0]*100}%\n")



In [ ]:
model=NeuralNet()

In [ ]:
model.train(train_input,train_target)

At iter:0,
loss =405.0003356933594 and
error_rate = 89.60000000000001%

At iter:100,
loss =146.4947509765625 and
error_rate = 9.8%

At iter:200,
loss =74.14320373535156 and
error_rate = 2.6%

At iter:300,
loss =38.056556701660156 and
error_rate = 0.0%

At iter:400,
loss =30.661540985107422 and
error_rate = 0.0%

At iter:500,
loss =30.256433486938477 and
error_rate = 0.0%

At iter:600,
loss =18.09187889099121 and
error_rate = 0.0%

At iter:700,
loss =18.30813980102539 and
error_rate = 0.0%

At iter:800,
loss =9.801643371582031 and
error_rate = 0.0%

At iter:900,
loss =10.203556060791016 and
error_rate = 0.0%

